## Atividade 1: Byte Pair Encoding

Esta atividade tem como objetivo a reprodução do algoritmo *Byte Pair Encoding* (BPE). O algoritmo BPE é utilizado na área de Processamento de Linguagem Natural para representar um grande vocabulário em um conjunto de trechos de palavras.

O algoritmo original segue os seguintes passsos:
- Dado um texto de entrada consideramos cada caractere do mesmo como uma unidade de token
- Verificamos então todas as combinações de adjacências no texto e computamos as frequências de adjacências iguais
- Mesclamos os caracteres que apresentam maior frequência de adjacência e consideramos essa mescla como um novo token no vocabulário
- Iteramos e repetimos o processo, até o critério estabelecido

Para este projeto foi utilizado um ambiente de execução Python 3.13 criado a partir do gerenciador Anaconda.

Se torna necessário também que o corpo disponibilizado esteja no mesmo diretório que este notebook na pasta corpus/ contendo os arquivos JSONs a serem analisados.



In [2]:
import os
import json
import io

In [3]:
# Aqui assumimos que o diretório com o corpo de arquivos JSON está no mesmo diretório que este notebook
directory = "corpus/"
count = 0

merged_text = (
    io.StringIO()
)  # Utilizando StringIO para melhorar a eficiência da concatenação
# Importação do corpo de texto em lote de acordo com o diretório
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)

        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

            count = count + 1
            # corpus_text = corpus_text + " " + data["text"] -> Resultado da leitura em mais de 3 minutos
            merged_text.write(
                data["text"] + " "
            )  # -> Leitura em 1:30 min e posteriormente mantém arquivos em memória

print(f"Arquivos lidos: {count}")

Arquivos lidos: 10000


In [4]:
corpus = merged_text.getvalue()

print(f"Tamanho total do texto lido: {len(corpus)}")

Tamanho total do texto lido: 71033261


Iremos aplicar o corpo de texto importado a nossa classe `BPETokenizer` criada no arquivo `tokenizer.py`, treinando o tokenizador com o texto fornecido e criando um vocabulário de tokens.
Esta classe utiliza dos métodos `count_pairs` e `merge_tokens` desenvolvidas no arquivo `helper_functions.py`. Estes auxiliam no processo de contagem de adjacências e junção de tokens respectivamente.

In [5]:
from helper_functions import count_pairs, merge_tokens

In [6]:
# Exemplificação do uso das funções
text_list = list("low, now, bow")
print(count_pairs(text_list))
# A sequência ow é a que mais se repete entre os atuais tokens do texto.

# Temos como parâmetros: a lista de tokens, a tupla com maior repetição
# E por fim como iremos chamar o novo token correspondente a tupla
print(merge_tokens(text_list, ("o", "w"), "Z"))

# Esperamos que de resultado ow seja substituído por Z em nosso texto de exemplo.

{('l', 'o'): 1, ('o', 'w'): 3, ('w', ','): 2, (',', ' '): 2, (' ', 'n'): 1, ('n', 'o'): 1, (' ', 'b'): 1, ('b', 'o'): 1}
['l', 'Z', ',', ' ', 'n', 'Z', ',', ' ', 'b', 'Z']


A classe `BPETokenizer` utiliza das funções acima iterando e criando um vocabulário, com os tokens previamente existentes e os gerados pelas junções, além de registrar o mapeamento de junções. Este vocabulário e mapeamento serão necessários para realizarmos a codificação e decodificação de um novo text que seja fornecido após o treinamento.

O tokenizador aqui criado tem como vocabulário inicial uma sequência de 256 tokens, representando as possibilidades de caractere em 1 byte a partir da codificação UTF-8. As junções então criadas serão postas após os 256 tokens originais. Isto também é importante para definirmos o critério de parada, ao declararmos nosso `BPETokenizer` devemos informar quantos laços de junção gostaríamos de realizar, ou seja a quantia de tokens final será 256 + [quantia de iterações].


In [7]:
from tokenizer import BPETokenizer

ATENÇÃO: A execução da próxima célula demanda um tempo de processamento considerável, em ambiente local teve um tempo de execução aproximado de 10 minutos. 

In [8]:
# Tokenizer com 276 tokens gerados.
num_merges = 20

bpe = BPETokenizer(num_merges)

bpe.train(corpus)

Tamanho do texto recebido em caracteres: 71033261
Tamanho do texto recebido em bytes: 73037124
Tamanho da lista de tokens após BPE: 57540928
Taxa de compressão (Tokens originais/Tokens BPE): 1.27X


O treinamento do tokenizador é passível de otimização, por meio de paralelização ou segmentação de chunks, uma vez que leva um tempo considerável para processar os milhões de caracteres inseridos. Para o propósito deste trabalho de manter o código simplificado, estas otimizações não foram realizadas.

In [9]:
# Salvamos os tokens gerados, temos que para o grande corpo de texto recebido, foi necessário 10 minutos em máquina local para processar 20 tokens.
bpe.save("voc")

256: [o][ ] -> [o ]
257: [a][ ] -> [a ]
258: [e][ ] -> [e ]
259: [s][ ] -> [s ]
260: [,][ ] -> [, ]
261: [d][e ] -> [de ]
262: [e][n] -> [en]
263: [m][ ] -> [m ]
264: [o][r] -> [or]
265: [e][r] -> [er]
266: [a][n] -> [an]
267: [a][r] -> [ar]
268: [e][s] -> [es]
269: [c][o] -> [co]
270: [.][ ] -> [. ]
271: [d][o ] -> [do ]
272: [o][s ] -> [os ]
273: [i][n] -> [in]
274: [a][l] -> [al]
275: [a][s ] -> [as ]


In [10]:
t = bpe.encode("Teste de codificação para BPE com 276 tokens")
print(t)

d = bpe.decode(t)
print(d)

[84, 268, 116, 258, 261, 269, 100, 105, 102, 105, 99, 97, 195, 167, 195, 163, 256, 112, 267, 257, 66, 80, 69, 32, 269, 263, 50, 55, 54, 32, 116, 111, 107, 262, 115]
Teste de codificação para BPE com 276 tokens


Para exemplificar a geração de um Tokenizador com mais tokens, vamos utilizar apenas os 100 primeiros arquivos JSON do corpo fornecido.

In [11]:
merged_text_100 = io.StringIO()

json_list = os.listdir(directory)
for filename in json_list[:100]:
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)

        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            merged_text_100.write(data["text"] + " ")

In [12]:
# Tokenizador com 512 tokens

num_merges = 512 - 256
bpe512 = BPETokenizer(num_merges)

corpus_100 = merged_text_100.getvalue()
bpe512.train(corpus_100)

Tamanho do texto recebido em caracteres: 486034
Tamanho do texto recebido em bytes: 498597
Tamanho da lista de tokens após BPE: 254527
Taxa de compressão (Tokens originais/Tokens BPE): 1.96X


In [13]:
bpe512.save("voc")

256: [o][ ] -> [o ]
257: [a][ ] -> [a ]
258: [e][ ] -> [e ]
259: [s][ ] -> [s ]
260: [,][ ] -> [, ]
261: [d][e ] -> [de ]
262: [e][r] -> [er]
263: [a][n] -> [an]
264: [e][n] -> [en]
265: [m][ ] -> [m ]
266: [o][r] -> [or]
267: [a][r] -> [ar]
268: [d][o ] -> [do ]
269: [e][s] -> [es]
270: [.][ ] -> [. ]
271: [o][s ] -> [os ]
272: [i][n] -> [in]
273: [a][l] -> [al]
274: [o][n] -> [on]
275: [a][s ] -> [as ]
276: [�][�] -> [ã]
277: [d][a ] -> [da ]
278: [i][c] -> [ic]
279: [en][t] -> [ent]
280: [s][t] -> [st]
281: [ã][o ] -> [ão ]
282: [�][�] -> [ç]
283: [a][t] -> [at]
284: [q][u] -> [qu]
285: [r][i] -> [ri]
286: [r][e] -> [re]
287: [�][�] -> [é]
288: [a][d] -> [ad]
289: [c][i] -> [ci]
290: [c][o] -> [co]
291: [e][l] -> [el]
292: [e][m ] -> [em ]
293: [e][s ] -> [es ]
294: [a][s] -> [as]
295: [a][m] -> [am]
296: [u][ ] -> [u ]
297: [�][�] -> [í]
298: [i][t] -> [it]
299: [r][o] -> [ro]
300: [a][i] -> [ai]
301: [�][�] -> [á]
302: [=][=] -> [==]
303: [d][i] -> [di]
304: [o][s] -> [os]
305: [*